
---

# 🐙 **Helm Template & Values (vLLM / TGI)**

---

## 🎯 What this chart does

* 🚀 Deploys **LLM inference servers** (vLLM or TGI)
* ⚡ Includes GPU requests, PVC for models, health probes
* 🌐 Exposes via **Service**, optional Ingress

---

## 📂 Chart layout

```
llm-inference/
  Chart.yaml
  values.yaml
  templates/
    deployment.yaml
    service.yaml
    pvc.yaml   # only if persistence.create=true
```

---

## ⚙️ Values (examples)

**vLLM**

```yaml
image: vllm/vllm-openai:latest
args: ["--model","/models/Llama-3-8B-Instruct","--max-num-seqs","8"]
resources: { requests: { cpu: 2, memory: 16Gi, nvidia.com/gpu: 1 } }
persistence: { create: false, existingClaim: models-rwx }
```

**TGI**

```yaml
image: ghcr.io/huggingface/text-generation-inference:latest
args: ["--model-id","/models/Llama-3-8B-Instruct","--max-batch-size","8"]
resources: { requests: { cpu: 2, memory: 16Gi, nvidia.com/gpu: 1 } }
persistence: { create: false, existingClaim: models-rwx }
```

---

## 📝 Templates (minimal)

**Deployment**

```yaml
containers:
  - name: api
    image: {{ .Values.image }}
    args: {{- toYaml .Values.args | nindent 10 }}
    ports: [{ containerPort: {{ .Values.port }} }]
    readinessProbe: { httpGet: { path: /health, port: {{ .Values.port }} } }
    volumeMounts:
      - { name: model, mountPath: /models, readOnly: true }
volumes:
  - name: model
    persistentVolumeClaim:
      claimName: {{ .Values.persistence.existingClaim }}
```

**Service**

```yaml
ports: [{ port: {{ .Values.service.port }}, targetPort: {{ .Values.port }} }]
type: {{ .Values.service.type }}
```

---

## 🚀 Install / Upgrade

```bash
helm install vllm ./llm-inference -n llm -f values.vllm.yaml
helm install tgi  ./llm-inference -n llm -f values.tgi.yaml
helm upgrade vllm ./llm-inference -n llm -f values.vllm.yaml
```

---

## 🧠 LLM Notes

* 📂 Mount `/models:ro` from PVC (RWX preferred).
* 🎛 Tune throughput flags first (vLLM `--max-num-seqs`, TGI `--max-batch-size`).
* 🔒 Use immutable image tags (avoid `:latest`).
* 📈 Add HPA/KEDA later for autoscaling.

---
